In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.05G/1.06G [00:08<00:00, 94.6MB/s]
100% 1.06G/1.06G [00:08<00:00, 130MB/s] 


In [4]:
import zipfile
zip_ref = zipfile .ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.src.utils.image_dataset import image_dataset_from_directory
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import numpy as np

In [6]:
# Load the dataset
image_size = (128, 128)  # Adjust size if needed
batch_size = 32

train_ds = image_dataset_from_directory(
    directory='/content/dogs_vs_cats/train',
    labels='inferred',
    label_mode='int',
    batch_size=batch_size,
    image_size=image_size
)

# Split the dataset into training and validation subsets
def split_dataset(dataset, validation_split=0.2):
    total_size = len(dataset)
    val_size = int(total_size * validation_split)
    train_size = total_size - val_size
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)
    return train_ds, val_ds

train_ds, validation_ds = split_dataset(train_ds)

# Normalize the images
def process(image, label):
    image = tf.cast(image / 255.0, tf.float32)
    return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)




Found 20000 files belonging to 2 classes.


In [ ]:
# Use a pre-trained model to extract features
pretrained_model = keras.applications.VGG16(include_top=False, input_shape=(128, 128, 3))
pretrained_model.trainable = False

# Define the feature extraction function
def extract_features(dataset):
    feature_list = []
    label_list = []
    for images, labels in dataset:
        features = pretrained_model.predict(images)
        features = features.reshape(features.shape[0], -1)
        feature_list.append(features)
        label_list.append(labels.numpy())
    return np.concatenate(feature_list), np.concatenate(label_list)

# Extract features and labels from the datasets
train_features, train_labels = extract_features(train_ds)
validation_features, validation_labels = extract_features(validation_ds)

# Split the extracted features into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=0)



1/1 [==============================] - 7s 7s/step


In [ ]:
#  Train an SVM classifier
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear'))
svm_model.fit(X_train, y_train)

# Evaluate the SVM model
test_score = svm_model.score(X_test, y_test)
print(f"SVM test accuracy: {test_score}")

# Print classification report
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image

# Load the pre-trained model for feature extraction
pretrained_model = keras.applications.VGG16(include_top=False, input_shape=(128, 128, 3))
pretrained_model.trainable = False

# Define the feature extraction function
def extract_features_from_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).resize((128, 128))
    image = np.array(image) / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Extract features using the pre-trained model
    features = pretrained_model.predict(image)
    features = features.reshape(features.shape[0], -1)  # Flatten features
    return features

# Load the external image
external_image_path = '/content/FELV-cat.jpg'  # Replace with the path to your external image
external_image_features = extract_features_from_image(external_image_path)

# Predict the class using the trained SVM model
predicted_class = svm_model.predict(external_image_features)
class_label = 'dog' if predicted_class[0] == 1 else 'cat'

print(f'The external image is classified as: {class_label}')
